# Cython

Cython is both a language (a superset of Python) and a Python library. 

With Cython, we start from a regular Python program and we add annotations about the type of the variables. Then, Cython translates that code to C and compiles the result to a Python extension module. Finally, we can use this compiled module in any Python program.

While dynamic typing comes with a performance cost in Python, statically-typed variables in Cython generally lead to faster code execution.

Performance gains are most signi cant in CPU-bound programs, notably in tight Python loops. By contrast, I/O-bound programs are not expected to bene t much from a Cython implementation.


In [ ]:
def fib(n):
    a,b = 1,1
    for i in range(n):
        a, b = a+b, a
    return a

In [ ]:
fib

In [ ]:
%load_ext cython

In [ ]:
%%cython
def cfib(int n):
    cdef int i, a, b
    a,b = 1,1
    for i in range(n):
        a, b = a+b, a
    return a

```python
def cfib(int n):
    cdef int i, a, b
    a,b = 1,1
    for i in range(n):
        a, b = a+b, a
    return a
```

* We added `int` to argument

* We used `cdef`

In [ ]:
%%cython
make an eror

In [ ]:
help("cdef")

In [ ]:
cfib

In [ ]:
fib(10)

In [ ]:
cfib(10)

In [ ]:
%%capture
# Number to compute
test_size = 100000
# Normal python
t1 = %timeit -n1 -r1 -o fib(test_size)
# Cython library
t2 = %timeit -n1 -r1 -o cfib(test_size)

In [ ]:
print("%sx speedup" % (t1.best // t2.best))

A C compiler is required.

# With numpy!

Generating the Mandelbrot fractal.

In [ ]:
import numpy as np

def mandelbrot_python(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i) 
            z= 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [ ]:
mandelbrot_python

In [ ]:
size = 200
iterations = 100

In [ ]:
%%timeit -n1 -r1 m = np.zeros((size, size),dtype=np.int32) 
mandelbrot_python(m, size, iterations)

In [ ]:
%%cython
import numpy as np
def mandelbrot_cython(int[:,::1] m, int size, int iterations):
    cdef int i, j, n
    cdef complex z, c

    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z= 0
            for n in range(iterations):
                if z.real**2 + z.imag**2 <= 100:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [ ]:
%%timeit -n1 -r1 m = np.zeros((size, size),dtype=np.int32) 
mandelbrot_cython(m, size, iterations)

# Wait. Does this really work?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

m = np.zeros((size, size),dtype=np.int32) 
mandelbrot_cython(m, size, iterations)
plt.imshow(np.log(m), cmap=plt.cm.hot)
plt.xticks([]); plt.yticks([])

All we have done is speci ed the type of the local variables and function arguments and bypassed NumPy's np.abs() function when computing the absolute value of z. These changes have helped Cython to generate more optimized C code from Python code.

# How it works

The cdef keyword declares a variable as a statically-typed C variable. C variables lead to faster code execution because the overhead from Python's dynamic typing is mitigated. Function arguments can also be declared as statically-typed C variables.
In general, variables used inside tight loops should be declared with cdef. To ensure that our code is well-optimized, we can use annotations. We just add the -a  ag after the %%cython magic and the non-optimized lines will be shown in a gradient of yellow (white lines are faster, yellow lines are slower). This is shown in the following screenshot. The color depends on the relative number of Python API calls at each line.

There are two ways of declaring NumPy arrays as C variables with Cython: using array buffers or using typed memory views. In this recipe, we used typed memory views. We will cover array buffers in the next recipe.
Typed memory views allow ef cient access to data buffers with a NumPy-like indexing syntax. For example, we can use int[:,::1] to declare a C-ordered 2D NumPy array with integer values, with ::1 meaning a contiguous layout in this dimension. Typed memory views can be indexed just like NumPy arrays.
However, memory views do not implement element-wise operations like NumPy. Thus, memory views act as convenient data containers within tight for loops. For element-wise NumPy-like operations, array buffers should be used instead.


We could achieve a signi cant performance speedup by replacing the call to np.abs with a faster expression. The reason is that np.abs is a NumPy function with a slight call overhead. It is designed to work with relatively large arrays, not scalar values. This overhead results in a signi cant performance hit in a tight loop such as here. This bottleneck can be spotted with Cython annotations.

Using Cython from IPython is very convenient with the %%cython cell magic. However, it is sometimes necessary to create a reusable C extension module with Cython. This is actually what IPython's %%cython cell magic does under the hood.

# Step by step

## Step 1

Write a standalone Cython script in a `.pyx` file. 

This should correspond exactly to the entire contents of a %%cython cell magic.


In [ ]:
%%writefile fib.pyx
def fcfib(int n):
    cdef int i, a, b
    a,b = 1,1
    for i in range(n):
        a, b = a+b, a
    return a

## Step 2

Create a setup.py file that we will use to compile the Cython module.


In [ ]:
%%writefile setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
    
setup(cmdclass = {'build_ext': build_ext},
      ext_modules = [Extension("cython_fibonacci", ["fib.pyx"])])

## Step 3

Execute this setup script with Python:

In [ ]:
! python setup.py build_ext --inplace

Two files have been created during the build process: 

1. the C source file 
2. and a compiled Python extension. 

The  le extension is .pyd on Windows (DLL files) and .so on UNIX

In [ ]:
! ls *.so

## Step 4

Finally, we can load the compiled module as usual (using from mandelbrot import mandelbrot).

In [ ]:
from cython_fibonacci import fcfib

We can use the optimized function

In [ ]:
fcfib(10)

In [ ]:
fcfib


With this technique, Cython code can also be integrated within a Python package. 

Here are a few references:

* Distributing Cython modules
       explained at http://docs.cython.org/src/userguide/source_files_and_compilation.html
* Compilation with Cython
       explained at http://docs.cython.org/src/reference/compilation.html

# Parallel

In [ ]:
! conda install -y ipyparallel

Now: open a terminal a create a cluster.

```bash
$ ipcluster start -n 4
```

In [ ]:
import os
import ipyparallel as ipp

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()